# 07. Corutines - a Lightweight Thread?

## What are corutines?
교재가 집필되던 시절과 지금의 라이브러리 버전이 상당히 차이가 난다는 점에 유의!

- https://kotlinlang.org/api/kotlinx.coroutines/kotlinx-coroutines-core/
- https://kotlinlang.org/api/latest/jvm/stdlib/kotlin.coroutines/


In [1]:
// 라이브러리 가져오는데 약간의 시간이 걸림
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.7.2")
import kotlinx.coroutines.*

In [2]:
GlobalScope.launch {
    launch {
        delay(500L)
        println("after delay " + Thread.currentThread().name)
    }

    println(Thread.currentThread().name)
    Thread.currentThread().join()
}

DefaultDispatcher-worker-1


StandaloneCoroutine{Active}@6348b9de

In [3]:
GlobalScope.launch {
    val job: Job = launch {
        delay(500L)
        println("after delay " + Thread.currentThread().name)
    }

    println(Thread.currentThread().name)
    job.cancel()
    Thread.currentThread().join()
}

after delay DefaultDispatcher-worker-2
DefaultDispatcher-worker-2


StandaloneCoroutine{Active}@48bdcb5a

In [4]:
/*여기?*/ runBlocking {
    val job: Job = launch {
        delay(500L)
        println("after delay " + Thread.currentThread().name)
    }

    println(Thread.currentThread().name)
    job.join()
}

Execution of code '/*여기?*/ runBlocking ...'
after delay Execution of code '/*여기?*/ runBlocking ...'


In [5]:
/*저기?*/ runBlocking {
    val job: Job = launch {
        delay(500L)
        println("coroutine!")
    }

    println("Hello, ")
    job.join()
}

Hello, 
coroutine!


## Coroutines and threads

In [6]:
GlobalScope.launch {
    val parentJob = Job()
    (1..30).forEach {
        launch(context = parentJob) {
            println( Thread.currentThread().name )
        }
    }
    joinAll(parentJob)
}

DefaultDispatcher-worker-4
DefaultDispatcher-worker-4
DefaultDispatcher-worker-4
DefaultDispatcher-worker-5
DefaultDispatcher-worker-6
DefaultDispatcher-worker-4
DefaultDispatcher-worker-5
DefaultDispatcher-worker-7
DefaultDispatcher-worker-9
DefaultDispatcher-worker-6
DefaultDispatcher-worker-8
DefaultDispatcher-worker-4
DefaultDispatcher-worker-6
DefaultDispatcher-worker-11
DefaultDispatcher-worker-10
DefaultDispatcher-worker-5
DefaultDispatcher-worker-7
DefaultDispatcher-worker-17
DefaultDispatcher-worker-9
DefaultDispatcher-worker-5
DefaultDispatcher-worker-6
DefaultDispatcher-worker-11
DefaultDispatcher-worker-10
DefaultDispatcher-worker-16
DefaultDispatcher-worker-15

StandaloneCoroutine{Active}@4ccef44f


DefaultDispatcher-worker-14
DefaultDispatcher-worker-13
DefaultDispatcher-worker-8
DefaultDispatcher-worker-12
DefaultDispatcher-worker-4


In [7]:
Runtime.getRuntime().availableProcessors()

20

In [8]:
val parallelism = max(1, Runtime.getRuntime().availableProcessors() - 1)

In [9]:
var sum = 0
// runBlocking { // single thread로 실행
GlobalScope.launch { // multi thread로 실행
    val parentJob = Job()
    (1..30).forEach {
        launch(context = parentJob) {
            for (i in 1..100) sum += 1
            val mysum = sum
            delay(10)
            println(mysum)
        }
    }
    delay(20)
    println("after: $sum")
    parentJob.children.forEach { it.join() }
}

200
100
300
400
600
500
900
800
700
1100
1000
1200
1300
1500
1400
1700
1600
1900
1800
2200
2300
2100
2000
3000
2900
2800
2700
2600
2500
2400
after: 3000


StandaloneCoroutine{Completed}@6974471f

In [ ]:
sum

## Coroutine examples

### Exception handling

In [10]:
suspend fun calculateValue() = withContext(context = Dispatchers.Default) { 
    // throw Exception() // 주석을 풀었다 걸었다 해보라
    999
}

val defaultValue = 1

runBlocking {
    launch {
        val result = try {
            calculateValue()
        } catch (exception: Exception) {
            defaultValue
        }
        print(result)
    }
}

999

StandaloneCoroutine{Completed}@196b5a88

### Resource releasing
코틀린이 문제가 아니라 Java 라이브리 자체가 버전이 올라가면서 바뀐 게 있어서 교재 코드는 작동안함

참고로 Java에서 실제로는 보통 이럴 때 try-with-resources 라고 부르는 형식의 try문을 쓰도록 권장

그리고 Kotin에서는 이런 경우 실제로 use라는 함수를 쓰도록 권장
- https://kotlinlang.org/api/latest/jvm/stdlib/kotlin.io/use.htmlhttps://kotlinlang.org/api/latest/jvm/stdlib/kotlin.io/use.html

### Non-cancelable block

In [11]:
runBlocking {
    val job = launch {
        try {
            delay(1000000)
        } finally {
            // withContext(NonCancellable) { // 이 부분의 주석을 풀어 보라
                try {
                    println("start")
                    delay(1000)
                    println("end")
                } catch (exception: Exception) {
                    println( exception )
                }
            // } /////////////////////////// // 이 부분의 주석을 풀어 보라 
        }
    }
    delay(300)
    job.cancel()
    job.join()
}

start
kotlinx.coroutines.JobCancellationException: StandaloneCoroutine was cancelled; job=StandaloneCoroutine{Cancelling}@51778078


### Parallel execution
이 예제는 정확히 말하자면 병렬(parallel) 실행이 아니라 비동기(asynchronous) 실행의 사례

In [12]:
import kotlin.random.*

Random.nextInt(100) // 여러 번 반복해 실행해 보라

88

In [13]:
import java.time.*

LocalDateTime.now()

2023-07-26T04:32:16.739307

In [14]:
class Image // dummy class

suspend fun downloadImage(): Image {
    delay( Random.nextInt(10) * 1000L )
    return Image()
}

fun displayImages(img1: Image, img2: Image) {
    println("$img1 ${LocalDateTime.now()}")
    println("$img2 ${LocalDateTime.now()}")
}

In [15]:
runBlocking {
    val job = Job().also { parentJob ->
        val deferred1 = async(parentJob) {
            println("Image 1 start dn ${LocalDateTime.now()}")
            downloadImage() 
        }.apply {
            invokeOnCompletion { println("Image 1 downloaded ${LocalDateTime.now()}") }
        }
        val deferred2 = async(parentJob) { 
            println("Image 2 start dn ${LocalDateTime.now()}")
            downloadImage() 
        }.apply {
            invokeOnCompletion { println("Image 2 downloaded ${LocalDateTime.now()}") }
        }
        displayImages(deferred1.await(), deferred2.await())
    }
    job.children.forEach { it.join() }
}

Image 1 start dn 2023-07-26T04:32:18.738234
Image 2 start dn 2023-07-26T04:32:18.747939
Image 1 downloaded 2023-07-26T04:32:21.749723
Image 2 downloaded 2023-07-26T04:32:26.753299
Line_13_jupyter$Image@6af4b32c 2023-07-26T04:32:26.755285
Line_13_jupyter$Image@6dd234a3 2023-07-26T04:32:26.783672


### Lazy executing

In [16]:
runBlocking {
    Job().also { parentJob ->
        val job = launch(context = parentJob, start = CoroutineStart.LAZY) { 
            println("Image start dn ${LocalDateTime.now()}")
            downloadImage()
        }
        /////
        println("다른 걸 하아아안~~~ ~~~ ~~~")
        delay(5000)
        println("~~~참 하다가 나중에 필요할 때")
        /////
        job.start()
    }.children.forEach { it.join() }
}

다른 걸 하아아안~~~ ~~~ ~~~
~~~참 하다가 나중에 필요할 때
Image start dn 2023-07-26T04:32:32.505754


### Channels
비동기 실행은 시간 차이를 두고 결과가 하나만 나오는데

일반적으로 여러 번에 걸쳐 여러 개가 나오는 것을 처리하려면 Channel을 쓰면 됨

multi-thread로 실행되는 맥락(context)까지 고려한다면 일반적으로 코루틴 사이에는 상태를 공유하는 것은 위험

그래서 Channel을 활용해 코루틴 사이에 안전하게 정보를 주고받을 수 있다 (즉, 코루틴 간의 동기화된 통신)

In [17]:
import kotlinx.coroutines.channels.*

runBlocking {
    val ch = Channel<Int>()
    launch {
        for (x in 1..10) {
            ch.send(x * x)
        }
    }
    repeat(10) {
        delay(500)
        println( ch.receive() )
    }
}

1
4
9
16
25
36
49
64
81
100


In [18]:
runBlocking {
    val ch = Channel<Int>()
    launch {
        for (x in 1..10) {
            delay(500)
            ch.send(x * x)
        }
    }
    repeat(10) {
        println( ch.receive() )
    }
}

1
4
9
16
25
36
49
64
81
100
